In [ ]:
! pip install git+https://github.com/dwavesystems/dwave-ocean-sdk

In [ ]:
def ising_extract_from_fasthare(filename):
    f = open(filename, "r")
    new_ising = {}
    while True:
        line = f.readline()
        #print(line)
        if line == '':
            break
        arr = line[0:-1].split(' ')
        if len(arr) == 3:
            new_ising[(int(arr[0]), int(arr[1]))] = -float(arr[2])
    return new_ising

In [ ]:
def bfs(state, target):
  check = {}
  for comp in G_metabolic.nodes:
    check[comp] = True
  queue = []
  for i in range(0, len(list_enzymes)):
    if state[i] == 1:
      queue.append(list_enzymes[i])
      check[list_enzymes[i]] = False
  
  while queue:
    head = queue.pop(0)
    list_successor = [i for i in G_metabolic.neighbors(head)]
    for comp in list_successor:
      if check[comp]:
        if comp in list_compounds:
          list_predecessor = [i for i in G_metabolic.predecessors(comp)]
          double_check = True
          for comp_pre in list_predecessor:
            if check[comp_pre]:
              double_check = False
              break
          if double_check:
            check[comp] = False
            queue.append(comp)
        else:
          check[comp] = False
          queue.append(comp)
  damage = 0
  removal = 0
  for comp in list_compounds:
    if comp in target:
      removal = removal + (check[comp] == False)
      #if check[comp]:
      #  print(comp)
    else:
      #if not check[comp]:
      #  print(comp)
      damage = damage + (check[comp] == False)
      
  return damage, removal == len(target)


In [ ]:
def di_initialize(list_enzymes, list_reactions, list_compounds, target):
  score_enzymes = {}
  state = [0]*len(list_enzymes)
  for i in range(0, len(list_enzymes)):
    state[i] = 1
    damage, res = bfs(state, target)
    state[i] = 0
    score_enzymes[list_enzymes[i]] = damage

  score_reactions = {}
  saved_reactions = {}
  for i in range(0, len(list_reactions)):
    damage = 100000000
    saved = []
    list_predecessor = [i for i in G_metabolic.predecessors(list_reactions[i])]
    for comp in list_predecessor:
      if comp in list_enzymes:
        if score_enzymes[comp] < damage:
          damage = score_enzymes[comp]
          saved = [comp]
    score_reactions[list_reactions[i]] = damage
    saved_reactions[list_reactions[i]] = saved

  score_compounds = {}
  saved_compounds = {}
  for i in range(0, len(list_compounds)):
    damage = 100000000
    saved = []
    list_predecessor = [i for i in G_metabolic.predecessors(list_compounds[i])]
    for comp in list_predecessor:
      saved = list(set(saved+saved_reactions[comp]))
    state = [0]*len(list_enzymes)
    for comp in saved:
      state[list_enzymes.index(comp)] = 1
    damage, res = bfs(state, target)
    if len(saved) != 0:
      score_compounds[list_compounds[i]] = damage
      saved_compounds[list_compounds[i]] = saved
    else:
      score_compounds[list_compounds[i]] = 100000000
      saved_compounds[list_compounds[i]] = saved
  
  return score_enzymes, score_reactions, saved_reactions, score_compounds, saved_compounds

In [ ]:
def di_main(list_enzymes, list_reactions, list_compounds, score_enzymes, score_reactions, saved_reactions, score_compounds, saved_compounds, target, iter_limit):
  iter = 0
  while iter < iter_limit:
    new_score_reactions = score_reactions.copy()
    new_saved_reactions = saved_reactions.copy()
    for i in range(0, len(list_reactions)):
      list_predecessor = [i for i in G_metabolic.predecessors(list_reactions[i])]
      for comp in list_predecessor:
        if comp in list_compounds:
          if score_compounds[comp] < new_score_reactions[list_reactions[i]]:
            new_score_reactions[list_reactions[i]] = score_compounds[comp]
            new_saved_reactions[list_reactions[i]] = saved_compounds[comp]

    new_score_compounds = score_compounds.copy()
    new_saved_compounds = saved_compounds.copy()
    for i in range(0, len(list_compounds)):
      saved = []
      list_predecessor = [i for i in G_metabolic.predecessors(list_compounds[i])]
      for comp in list_predecessor:
        saved = list(set(saved+new_saved_reactions[comp]))
      
      state = [0]*len(list_enzymes)
      for comp in saved:
        state[list_enzymes.index(comp)] = 1
      damage, res = bfs(state, target)

      if damage < new_score_compounds[list_compounds[i]]:
        new_score_compounds[list_compounds[i]] = damage
        new_saved_compounds[list_compounds[i]] = saved
    
    check_stop = True
    for comp in list_reactions:
      if new_score_reactions[comp] != score_reactions[comp]:
        check_stop = False
        break

    for comp in list_compounds:
      if new_score_compounds[comp] != score_compounds[comp]:
        check_stop = False
        break
    
    if check_stop:
      break
    else:
      score_reactions = new_score_reactions.copy()
      saved_reactions = new_saved_reactions.copy()
      score_compounds = new_score_compounds.copy()
      saved_compounds = new_saved_compounds.copy()
      iter = iter + 1
  return score_enzymes, score_reactions, saved_reactions, score_compounds, saved_compounds, iter

In [ ]:
def di_extract_results(list_enzymes, saved_compounds, target):
  removed_enzymes = []
  for comp in target:
    removed_enzymes = removed_enzymes + saved_compounds[comp]
  removed_enzymes = list(set(removed_enzymes))

  bits = [0]*len(list_enzymes)
  for i in range(0, len(list_enzymes)):
    if list_enzymes[i] in removed_enzymes:
      bits[i] = 1
  damage, res = bfs(bits, target)
  return damage, res

In [ ]:
import random

def di_generate_target(sz, list_compounds):
  if sz >= len(list_compounds):
    return []

  return random.sample(list_compounds, sz)

In [ ]:
def bigger(k1, k2):
  if type(k1) is type(k2):
    return k1 > k2
  else:
    if type(k1) is int:
      return False
    else:
      return True

def extract_quadratic_func(input_comp):
  temp_comp = {}
  keys = input_comp.keys()
  for k in keys:
    if type(k) is not int:
      temp_comp[k] = input_comp[k]*input_comp[k]
  for k1 in keys:
    for k2 in keys:
      if (k1 != k2) and (bigger(k1, k2)):
        if type(k1) is int:
          if type(k2) is not int:
            temp_comp[k2] = temp_comp[k2] + 2*input_comp[k1]*input_comp[k2]
        else:
          if type(k2) is int:
            temp_comp[k1] = temp_comp[k1] + 2*input_comp[k2]*input_comp[k1]
          else:
            if (k1, k2) not in temp_comp.keys():
              temp_comp[(k1, k2)] = 2*input_comp[k1]*input_comp[k2]
            else:
               temp_comp[(k1, k2)] = temp_comp[(k1, k2)] + 2*input_comp[k1]*input_comp[k2]
  return temp_comp

def combine_comp(qubo_comp, temp_comp, ins):
  keys = temp_comp.keys()
  for k in keys:
    if k not in qubo_comp:
      qubo_comp[k] = temp_comp[k] * ins
    else:
      qubo_comp[k] = qubo_comp[k] + temp_comp[k] * ins
  return qubo_comp

In [ ]:
def qubo_initialize(list_enzymes, list_reactions, list_compounds, target):
  C = target
  qubo_comp = {}
  A1 = 1
  A2 = 300
  A3 = 250
  A4 = 250
  coefficient = 0
  for comp in list_compounds:
    if comp not in C:
      qubo_comp[comp] = A1
    else:
      print
      qubo_comp[comp] = -A2
      coefficient = coefficient + A2

  for comp_i in list_compounds:
    list_predecessor = [i for i in G_metabolic.predecessors(comp_i)]

    for comp_j in list_predecessor:
      temp_comp = extract_quadratic_func({comp_i: 1, comp_j: -1})
      qubo_comp = combine_comp(qubo_comp, temp_comp, A3)
    
    for comp_j in list_predecessor:
      temp_comp = extract_quadratic_func({comp_i: 1, comp_j: -1, 1:1})
      qubo_comp = combine_comp(qubo_comp, temp_comp, A3)
      #coefficient = coefficient + A3
    
    input_comp = {comp_i: -1, 1:1}
    for comp_j in list_predecessor:
      input_comp[comp_j] = 1
    
    for i in range(0, len(list_predecessor) + 1):
      input_comp['W_'+str(comp_i)+'_'+str(i)] = -i
    
    temp_comp = extract_quadratic_func(input_comp)
    qubo_comp = combine_comp(qubo_comp, temp_comp, A3)
    coefficient = coefficient + A3

    input_comp = {1:1}
    for i in range(0, len(list_predecessor) + 1):
      input_comp['W_'+str(comp_i)+'_'+str(i)] = -1
    
    temp_comp = extract_quadratic_func(input_comp)
    qubo_comp = combine_comp(qubo_comp, temp_comp, A3)
    coefficient = coefficient + A3

  for comp_i in list_reactions:
    list_predecessor = [i for i in G_metabolic.predecessors(comp_i)]
    removal_list = []
    if is_revertable[comp_i]:
      for comp_j in list_predecessor:
        if comp_j in list_compounds:
          removal_list.append(comp_j)
    for comp_j in removal_list:
      list_predecessor.remove(comp_j)
    for comp_j in list_predecessor:
      temp_comp = extract_quadratic_func({comp_i: 1, comp_j: -1})
      qubo_comp = combine_comp(qubo_comp, temp_comp, A4)
    
    for comp_j in list_predecessor:
      temp_comp = extract_quadratic_func({comp_i: 1, comp_j: -1, 1:-1})
      qubo_comp = combine_comp(qubo_comp, temp_comp, A4)
      #print(comp_i, temp_comp)
      #coefficient = coefficient - A4
    
    input_comp = {comp_i: 1}
    for comp_j in list_predecessor:
      input_comp[comp_j] = -1
    
    for i in range(0, len(list_predecessor) + 1):
      input_comp['T_'+str(comp_i)+'_'+str(i)] = i
    
    temp_comp = extract_quadratic_func(input_comp)
    qubo_comp = combine_comp(qubo_comp, temp_comp, A4)

    input_comp = {1:1}
    for i in range(0, len(list_predecessor) + 1):
      input_comp['T_'+str(comp_i)+'_'+str(i)] = -1
    
    temp_comp = extract_quadratic_func(input_comp)
    qubo_comp = combine_comp(qubo_comp, temp_comp, A4)
    coefficient = coefficient + A4
  return qubo_comp, coefficient

In [ ]:
from dimod import BinaryQuadraticModel
from dwave.system import LeapHybridSampler

def qubo_splitting(qubo_comp):
  linear_comp = {}
  quadratic_comp = {}
  for k in qubo_comp.keys():
    if type(k) is str:
      linear_comp[k] = qubo_comp[k]
    else:
      quadratic_comp[k] = qubo_comp[k]
  return linear_comp, quadratic_comp

def qubo_main(linear_comp, quadratic_comp, coefficient, target, time_limit_):
  bqm = BinaryQuadraticModel(linear_comp,
                           quadratic_comp,
                           coefficient,
                           'BINARY')
  sampler = LeapHybridSampler()
  sampleset = sampler.sample(bqm,time_limit=time_limit_)
  print(sampleset.lowest(atol=.5))

  removed_enzymes_2 = []
  for comp in list_enzymes:
    if sampleset.first.sample[comp] == 1:
      removed_enzymes_2.append(comp)
  
  bits_2 = [0]*len(list_enzymes)
  for i in range(0, len(list_enzymes)):
    if list_enzymes[i] in removed_enzymes_2:
      bits_2[i] = 1
  
  damage, res = bfs(bits_2, target)
  return damage, res

In [ ]:
#self generate targets

import networkx as nx
from networkx.algorithms.shortest_paths.unweighted import predecessor
import dimod
import time
from subprocess import call

pathway_list = ['Purine', 'Carbon metabolism', 'Nucleotide metabolism', 'Biosynthesis of amino acids']
species_name = 'hsa'
data_type = 'large'
time_limit = 1200

test_iterations = 2
test_step = 5
iter_limit = 20
    
for pathway_name in pathway_list:
    list_enzymes = []
    list_reactions = []
    list_compounds = []
    G_metabolic = nx.DiGraph()
    nodes = []
    edges = []
    mode = 0
    with open('large_data/'+species_name+'_filtering/'+pathway_name+'.input') as f:
        while mode <= 3:
            if mode < 3:
                num_nodes = int(f.readline())
                #print(num_nodes)
                for i in range(0, num_nodes):
                    element = f.readline()
                    #print(element)
                    nodes.append(element[0:-1])
                    if mode == 0:
                        list_enzymes.append(element[0:-1])
                    elif mode == 1:
                        list_reactions.append(element[0:-1])
                    else:
                        list_compounds.append(element[0:-1])
                mode = mode + 1
            else:
                num_edges = int(f.readline())
                for i in range(0, num_edges):
                    line = f.readline()
                    element = line[0:-1].split(' ')
                    edges.append(element)
                mode = mode + 1
    G_metabolic.add_edges_from(edges)
 
    list_removal = []
    for comp in G_metabolic.nodes:
        list_predecessor = [i for i in G_metabolic.predecessors(comp)]
        if comp in list_compounds and len(list_predecessor) == 0:
            list_removal.append(comp)
            list_compounds.remove(comp)
    G_metabolic.remove_nodes_from(list_removal)

    is_revertable = {}
    for comp in list_reactions:
        list_predecessor = [i for i in G_metabolic.predecessors(comp)]
        list_successor = [i for i in G_metabolic.neighbors(comp)]
        check = False
        for comp_2 in list_predecessor:
            if comp_2 in list_successor:
                check = True
                break
        is_revertable[comp] = check
    
    sz = 1
    testcase = []
    while sz < len(list_compounds):
        for it in range(0, test_iterations):
            record = {}
            target = di_generate_target(sz, list_compounds)
            if len(target) == 0:
                break
            time_start = time.time()
            score_enzymes, score_reactions, saved_reactions, score_compounds, saved_compounds = di_initialize(list_enzymes, list_reactions, list_compounds, target)

            score_enzymes, score_reactions, saved_reactions, score_compounds, saved_compounds, iter = di_main(list_enzymes, list_reactions, list_compounds, score_enzymes, score_reactions, saved_reactions, score_compounds, saved_compounds, target, iter_limit)

            damage, check = di_extract_results(list_enzymes, saved_compounds, target)
            time_end = time.time()

            record['Target_size'] = sz
            record['Target'] = target
            record['DI_damage'] = damage
            record['DI_running_time'] = time_end - time_start
            
            
            testcase.append(record)
        sz = sz + test_step
    
    for record in testcase:
        target = record['Target']
        qubo_comp, coefficient = qubo_initialize(list_enzymes, list_reactions, list_compounds, target)

        linear_comp, new_qubo = qubo_splitting(qubo_comp)
        bqm = BinaryQuadraticModel(linear_comp,
                                   new_qubo,
                                   coefficient,
                                   'BINARY')

        ising_model = bqm.to_ising()
        
        total_nodes = 0
        dict_nodes = {}
        dict_edges = {}

        linear_keys = ising_model[0].keys()
        for k in linear_keys:
            dict_nodes[k] = total_nodes
            total_nodes = total_nodes + 1

        quadratic_keys = ising_model[1].keys()
        for k in quadratic_keys:
            if ising_model[1][k] != 0:
                node_1 = k[0]
                node_2 = k[1]
                dict_edges[(dict_nodes[node_1], dict_nodes[node_2])] = ising_model[1][k]


        f = open("large_data/"+species_name+"_label_fasthare/"+pathway_name+".txt", "w")

        nd_keys = dict_nodes.keys()
        ed_keys = dict_edges.keys()

        countt = 0
        for k in nd_keys:
            if ising_model[0][k] != 0:
                countt = countt + 1

        f.write(str(len(nd_keys)+1)+' '+str(len(ed_keys)+countt)+'\n')
        for k in ed_keys:
            node_1 = k[0]
            node_2 = k[1]
            weight = dict_edges[k]
            #print(str(node_1)+' '+str(node_2)+' '+str(weight)+'\n')
            f.write(str(node_1)+' '+str(node_2)+' '+str(weight)+'\n')

        for k in nd_keys:
            node_1 = dict_nodes[k]
            node_2 = len(nd_keys)
            weight = ising_model[0][k]
            if weight == 0:
                continue
            f.write(str(node_1)+' '+str(node_2)+' '+str(weight)+'\n')
        f.close()
        
        
        command = ['python3','FastHare/parse.py','large_data/'+species_name+'_label_fasthare/'+pathway_name]
        call(command)
        command = ['./FastHare/fasthare','large_data/'+species_name+'_label_fasthare/'+pathway_name+'.net', 'large_data/'+species_name+'_label_fasthare/'+'output', '2', '0']
        call(command)

        filename = 'large_data/'+species_name+'_label_fasthare/'+'output_compress'
        new_quadratic = ising_extract_from_fasthare(filename)
        linear_comp = {}
        #break
        new_bqm = dimod.BinaryQuadraticModel.from_ising({}, new_quadratic, offset = ising_model[2])
        #new_bqm = dimod.BinaryQuadraticModel.from_ising(ising_model[0], ising_model[1], offset = ising_model[2])

        start = time.time()
        sampler = LeapHybridSampler()
        sampleset = sampler.sample(new_bqm,time_limit=time_limit)
        end = time.time()
        print(sampleset.lowest(atol=.5))
        end = time.time()
        
        filename = 'large_data/'+species_name+'_label_fasthare/'+pathway_name+'.net'
        new_quadratic = ising_extract_from_fasthare(filename)
        linear_comp = {}
        old_bqm = dimod.BinaryQuadraticModel.from_ising({}, new_quadratic, offset = ising_model[2])
        
        samples = {}
        for sample in sampleset.samples():   
            samples = sample

        filename = 'large_data/'+species_name+'_label_fasthare/'+'output_flip'
        state_ = {}
        curr_idx = 0
        f = open(filename, "r")
        new_ising = {}
        line = f.readline()
        arr = line[0:-2].split(' ')
        new_n = int(arr[0])
        old_n = int(arr[1])

        line = f.readline()
        check_flip = line[0:-2].split(' ')
        for i in range(0, new_n):
            line = f.readline()
            if line == '':
                break
            #print(line)
            arr = line[0:-2].split(' ')
            for i in range(1, len(arr)):
                node = int(arr[i])
                if check_flip[node] == '1':
                    #print(node)
                    state_[node] = - samples[curr_idx]
                else:
                    state_[node] = samples[curr_idx]
            curr_idx = curr_idx + 1
        
        f = open("large_output/"+species_name+"/"+pathway_name+".out", "a")
        f.write('Target_size\n')
        f.write(str(record['Target_size'])+'\n')
        f.write('Target\n')
        for enzy in record['Target']:
            f.write(enzy+'\n')
        f.write('DI_damage\n')
        f.write(str(record['DI_damage'])+'\n')
        f.write('DI_running_time\n')
        f.write(str(record['DI_running_time'])+'\n')
        f.write('QC_damage\n')
        f.write(str(old_bqm.energy(state_)) + '\n')
        f.write('QC_QPU_access_time\n')
        f.write(str(sampleset.info['qpu_access_time']/1000000)+'\n')
        f.write('QC_running_time\n')
        f.write(str(sampleset.info['run_time']/1000000)+'\n')
        f.close()